In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import statistics

from konlpy.tag import Twitter
from konlpy.utils import pprint
twitter = Twitter()
from collections import Counter
from collections import namedtuple

pd.set_option('max_colwidth', 500)

common_front = '../../dataset/go_0715/gogosing_' 
common_back = '.json'

def get_file(file_num):   
    file_path = common_front + str(file_num) + common_back
    json_data = open(file_path).read()
    data = json.loads(json_data)
    df_data = pd.DataFrame(data)
    return df_data


# 74번 파일이 에러가 나서 제외함
def concat_file(file_num):
    start_file = pd.DataFrame(get_file(1))
    for i in range(2,file_num+1):
        if( i == 74):
            continue
        df_tmp = pd.DataFrame(get_file(i))
        start_file = pd.concat([start_file, df_tmp])
    start_file = start_file.reset_index(drop=True)
    return start_file

In [19]:
go_data = concat_file(156)

# apply to gogosing data

In [20]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from konlpy.tag import Twitter

/usr/local/lib/python2.7/dist-packages/scipy/sparse/lil.py:16: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._traversal import breadth_first_order, depth_first_order, \
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/__init__.py:169: R

In [21]:
num_cId = go_data['cId'].value_counts()
num_cId = pd.DataFrame(num_cId)
print(num_cId)

                      cId
네이버 페이 구매자          52539
rkdldk5**             475
yeji04**              449
elqxhs**              339
dnv**                 314
rladmswl9708**        308
wlgus5**              305
pp**                  301
wertyuio9**           292
sumin12**             278
dldmsql10**           276
kma05**               271
dream3**              245
rudgml72**            244
soa12**               237
kimhani3**            210
45216058**            209
sjj44**               197
rlaqhfk5**            193
tladnjs2**            191
tpwls12**             190
jy03**                189
egll**                188
jeje05**              187
bha11**               186
hooming**             186
tofha40**             186
ds357**               185
miliastla**           183
phr9101**             183
...                   ...
유민지                     1
461301292**             1
gook**                  1
byeolttar**             1
1698601763758607**      1
tjddlaks1**             1
eofhdrk**   

In [47]:
cIds = num_cId.drop(num_cId.index[0])  #drop naverpay
cIds = cIds[cIds['cId']!=1]  #drop count1
cIds = cIds[cIds['cId']!=2]  #drop count2
cIds_list = cIds.index.tolist()
len(cIds_list)

68330

In [48]:
def tokenizer_twitter_morphs(id):
    df = (go_data.loc[go_data['cId']==id, ['cId','desc']])
    tagged_data = [TaggedDocument(words= twitter.morphs(_d), tags=[str(i)]) for i, _d in enumerate(df['desc'])]
    return tagged_data

In [49]:
r = tokenizer_twitter_morphs('psbp**')
r = pd.DataFrame(r)
r

,words,tags
0,"[조금, 긴, 슬랙스, 를, 찾다, 가, 고고싱, 신상, 에, 올라왔, 길래, 바로, 주문했, 어요, 평소, 26, 입는, 데, s, 사이즈, 조금, 헐렁한데, 괜찮, 아요, 근데, 이렇게, 얇은, 재질, 인, 줄, 은, 몰랐, 는데, .., 여름, 에도, 입, 을, 수, 있겠, 어요, ~~]",[0]
1,"[데, 일리, 로, 입, 을, 흰, 블라우스, 찾다, 가, 샀, 어요, !, 저렴, 한, 가격, 에, 잘, 산, 거, 같아, 요, 스, 판도, 좋, 아서, 불편하지, 않아, 요, :)]",[1]
2,"[흰색, 나시, 가, 필요해서, 다른, 옷, 살때, 나시, 도, 같이, 샀, 어요, ~, 이름, 처럼, 스판, 쫀쫀, 해서, 좋, 아요, ㅎㅎㅎ]",[2]
3,"[예뻐, 서, 샀, 는데, 차라리, 다른, 걸, 살걸, 그랬, 네요, ,,, 니트, 넣어, 서, 입었, 는데, 뚱뚱해, 보여, 요, 치마, 길이, 도, 앞, 에가, 조금, 더, 길, 어요, 벨트, 도, 다른, 걸, 써, 야할, 것, 같아, 요]",[3]
4,"[배송, 빨라, 요, ~, 옷, 은, 예쁜, 데, 겨울, 에, 입기, 에는, 조금, 얇아, 요, ., 색도, 사진, 이랑, 똑같, 아요, 완전, 흰, 바지, 보다, 아이보리, 라, 마음, 에, 들어, 요]",[4]
5,"[배송, 도, 이틀, 만에, 오고, 옷, 도, 예뻐, 요, ., 브라운, 이랑, 블랙, 고민했, 는데, 마음, 에, 들어, 요, ㅋㅋ, 팔, 은, 좀, 길, 어서, 한번, 더, 접어, 입는, 게, 더, 예뻐, 요]",[5]
6,"[비슷한, 신발, 이, 있었, 는데, 230, 신다, 가, 조금, 널널, 하게, 신고, 싶, 어서, 235, 로, 주문했, 는데, 진짜, 가볍, 고, 가격, 대비, 만족해, 요, 냄새, 도, 별로, 안나, 요]",[6]
7,"[편한, 옷, 에, 맬, 가방, 이, 없어, 서, 찾던, 중, 에, 예쁜, 가방, 을, 찾았, 어요, ., 배송, 은, 삼일, 만에, 왔, 고, 처음, 받았, 을, 때, 는, 먼지, 가, 조금, 있었, 는데, ,, 괜찮, 았, 어요]",[7]
8,"[포토, 리뷰, 1, 등, 이에요, :), 원래, 박, 시한, 옷, 을, 좋아해서, 시켰, 는데, 조금, 더, 길고, 소매, 도, 길었, 으면, 더, 예뻤, 을텐데, 옷, 은, 예뻐, 요, ., 소매, 랑, 목, 이, 너무, 얇아, 서, 금방, 늘어날, 거, 같아, 요]",[8]
9,"[가격, 대비, 좋, 아요, 기장, 이, 조금, 더, 길었, 으면, 좋, 았, 을텐데, 팔, 길이, 는, 두, 번, 접어, 입으, 면, 예뻐, 요, ㅋㅋ, 색상, 도, 사진, 이랑, 똑같, 아서, 마음, 에, 들어, 요]",[9]


In [50]:
r1 = tokenizer_twitter_morphs(cIds_list[1])
r1 = pd.DataFrame(r1)
r1

,words,tags
0,"[다른, 사이트, 보다, 저렴, 해, 요, !!, 옐로우, 이뻐, 요, 린넨, 이라, 시원하구, 요, 근데, 앞, 에, 많이, 파, 여서, 잘, 입어, 야, 해, 요크, 롭, 기장, 이라, 키, 크, 신, 분, 들, 은, 잘, 생각하고, 구매, 하세, 용]",[0]
1,"[음, ..., 이쁘, 긴, 이뻐, 요, 아무리, 린넨, 이라, 지만, 옐로우, 에, 비해서, 보풀, 이, 너무, 많이, 일어나, 있어, 서, ..., 보풀, 제거, 먼저, 했, 네요, ㅠ, 입으, 면, 이뻐, 요, 근데, 기장, 이, 아주아주, 짧은, 크, 롭, 느낌, 이라, 키, 크, 신, 분, 들, 은, 주의, 를, ...]",[1]
2,"[이, 게, 여름, 용인, 지, .., 린넨, 인지, ..., 구분, 이, 안, 갈, 정도, 로, 두꺼워, 요, ,., 여름, 엔, 무리, 일, 듯, 싶어, 요, 가을, ??, 정도, 면, 괜찮, 겠, 네요, 약간, 무게, 감도, 있어, 서, 다크, 그린, 색, 은, 맘, 에, 들, 구요, 디자인, 도, 괜찮, 아요, 가을, 에, 입, 을, 게요]",[2]
3,"[오, 좋, 아, 좋, 아, ㅋㅋ, 배송비, 채우려, 고, 구매, ㅎ, 한, 건데, 자석, 너무, 편하고, 패턴, 단순해서, 심플, 한, 게, 맘, 에, 들어, 요, 괜찮, 아서, 블랙, 도, 사려, 구요, 엄마, 도, 사, 드릴, 거, 에, 용]",[3]
4,"[음, !!, 이쁘, 네요, !!, 그냥, 매일, 고민, 만, 했, 었, 는데, ㅋㅋㅋ, 할인, 하는, 김, 에, 질렀는데, 잘, 샀, 어요, 조금, 파, 이, 긴, 했, 어도, 이뻐, 요, !!!, 이쁜, 민트, 색, 이, 에, 용, ㅎㅎㅎ]",[4]
5,"[색깔, 이, 화면, 보다, 많이, 어둡, 네요]",[5]
6,"[아이보리, 품절, 인게, 너무, 아쉬운, ㅠ, 기장, 이, 너무, 짧긴, 해도, 이, 너, 로, 입으, 면, 넥, 라, 이인, 이, 이뻐, 서, 좋, 아요, !, 몸, 에, 딱, 핏되는, 게, 날씬, 해, 보이, 구, .., 대신, 배, 에, 힘, 뽝, 줘, 야하는, ㅋㅋㅋㅋ, 근데, 맘에듭니, 다, !]",[6]
7,"[손바닥, 만, 한, 천, 쪼가리, 가, 와, 서, 너무, 당황, ;;;, 엄청, 쫀존, 해서, 몸, 에, 들어가긴, 해, 요, ㅋㅋㅋ, 근데, 기장, 이, 조금, 더, 길었, 으면, 하는, 아쉬움, 이, ..., 배꼽, 티, 같아, 요, ㅠㅠㅠㅠ, 모델, 처럼, 넣어, 서, 입어, 야, 겠, 네요, !, ㅎㅎ]",[7]
8,"[핑크, 색상, 도, 은은하니, 이쁘, 구요, ~, 팔뚝, 은, 좀, 끼는, 데, 괜찮, 아요, 몸매, 가, 고스, 란, 히, 드러나서, 날씬, 한, 분들, 입으, 면, 진짜, 이쁠, 거, 같아, 요, 기본, 으로, 너무, 좋, 고, 여기저기, 매치, 하기, 편안하니, 넘, 좋, 네, 용옷, 도, 편해, 요, !]",[8]
9,"[첫, 포토, 리뷰, 에요, !!!, 옐로우, 먼저, 받았, 는, 데, 용, ~, 길이, 가, 엄청, 기네, 요, ...., 키, 가, 작아, 서, 완전, 미니, 원피스, 에요, ㅋㅋㅋ, 그리고, 여름, 에, 못, 입어, 요, 더워, 요, 밤, 에, 추, 울, 때, 걸치, 면, 몰라, 도, 지금, 덥, 네요, !!!, 그래, 두, 이브, 니, 가, ~, 가을, 에, 입, 을, 게요]",[9]


In [51]:
def training_pvdbow(tagged_data):
    max_epochs = 100
    vec_size = 20
    alpha = 0.025
    
    model = Doc2Vec(size=vec_size,      # Dimensionality of the feature vectors
                    window=8,            # The maximum distance 
                    alpha=alpha,        # initial learning-rate
                    min_alpha=0.00025,  # min learning-rate
                    min_count=1,        # Ignores all words with total frequency lower than this.
                    dm =0)              # pv-dbow

    model.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        #print('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha


    print("Model Saved")
    
    return model

In [52]:
def similarity(model):
    sm = []
    for i in range(len(model.docvecs)-1):
        for j in range(i+1, len(model.docvecs)):
            sm.append(model.docvecs.similarity(i,j))
        
    print str('최대값: ') + str(max(sm))
    print str('상위25%: ') + str(np.percentile(sm,75))
    print str('중간값: ') + str(np.percentile(sm,50))
    print str('상위75%: ') + str(np.percentile(sm,25))
    print str('평균: ') + str(np.mean(sm))
    
    return sm

In [53]:
 # 소수점 아래 3자리까지 나타내기, 4째자리에서 반올림
def short_float(val):
    value = float("{:.4f}".format(val))
    return value


In [54]:
def print_want_val(sm_list,used_func):
    val=(used_func(sm_list))
    val=short_float(val)
    
    return val

In [55]:
dk = [2.5,8.36,4.2795]
np.mean(dk)

5.046499999999999

# result analysis

In [56]:
r1 = pd.read_excel('twitter_result1.xlsx', sheetname='Sheet1')

In [57]:
r2 = pd.read_excel('twitter_result2.xlsx', sheetname='Sheet1')

In [58]:
r3 = pd.read_excel('twitter_result3_new.xlsx', sheetname='Sheet1')

In [59]:
r1

,cId,max,q3,mid,q1,mean
0,rkdldk5**,0.9997,0.4266,0.3133,0.2057,0.318713
1,yeji04**,0.9996,0.4360,0.3272,0.2231,0.331217
2,elqxhs**,0.9997,0.4145,0.3037,0.1981,0.308584
3,dnv**,0.9996,0.4257,0.3165,0.2118,0.320915
4,rladmswl9708**,0.8789,0.4181,0.3095,0.2030,0.312666
5,wlgus5**,0.9994,0.4299,0.3204,0.2186,0.326358
6,pp**,0.9992,0.4235,0.3157,0.2101,0.319019
7,wertyuio9**,0.9994,0.4298,0.3213,0.2187,0.325979
8,sumin12**,0.9999,0.4878,0.4196,0.3215,0.440175
9,dldmsql10**,0.9994,0.4170,0.3059,0.2013,0.311812


In [60]:
r2

,cId,max,q3,mid,q1,mean
0,jupark05**,0.8964,0.4038,0.3270,0.3078,0.395174
1,su06**,0.4261,0.3684,0.3387,0.2976,0.336207
2,heidy**,0.4459,0.3936,0.3152,0.3012,0.341578
3,fnskdpsk91**,0.4120,0.3555,0.3370,0.3270,0.340414
4,gusdk1004**,0.5075,0.4312,0.3898,0.3389,0.382159
5,wjdals54**,0.4271,0.3628,0.3491,0.3391,0.354217
6,kjy09**,0.4385,0.3700,0.3268,0.3117,0.340672
7,이서연,0.4430,0.3932,0.3513,0.3284,0.360034
8,alswl99**,0.3653,0.3560,0.3472,0.3376,0.347081
9,with12**,0.4011,0.3494,0.3372,0.3198,0.336305


In [61]:
r3

,cId,max,q3,mid,q1,mean
0,fneldk**,0.2696,0.2594,0.2492,0.2362,0.247353
1,dmstj52**,0.4306,0.3363,0.2419,0.1996,0.276612
2,chfhd19**,0.3053,0.2868,0.2682,0.2391,0.261174
3,yunkss**,0.9710,0.9517,0.9323,0.9305,0.943977
4,wlk2**,0.4654,0.3716,0.2779,0.2126,0.296876
5,jijunli**,0.3734,0.2961,0.2188,0.2118,0.265638
6,tato**,0.2911,0.2755,0.2599,0.2560,0.267721
7,ggovl**,0.3620,0.2844,0.2067,0.2002,0.254129
8,zim**,0.4942,0.3507,0.2072,0.1843,0.287630
9,yuri46**,0.3374,0.3060,0.2746,0.2362,0.269963


In [62]:
total_df = pd.concat([r1, r2, r3], ignore_index=True)

In [63]:
total_df.rename(columns={"max":"_max"}, inplace = True)
total_df

,cId,_max,q3,mid,q1,mean
0,rkdldk5**,0.9997,0.4266,0.3133,0.2057,0.318713
1,yeji04**,0.9996,0.4360,0.3272,0.2231,0.331217
2,elqxhs**,0.9997,0.4145,0.3037,0.1981,0.308584
3,dnv**,0.9996,0.4257,0.3165,0.2118,0.320915
4,rladmswl9708**,0.8789,0.4181,0.3095,0.2030,0.312666
5,wlgus5**,0.9994,0.4299,0.3204,0.2186,0.326358
6,pp**,0.9992,0.4235,0.3157,0.2101,0.319019
7,wertyuio9**,0.9994,0.4298,0.3213,0.2187,0.325979
8,sumin12**,0.9999,0.4878,0.4196,0.3215,0.440175
9,dldmsql10**,0.9994,0.4170,0.3059,0.2013,0.311812


In [64]:
total_df._max.describe()

count    68330.000000
mean         0.487768
std          0.184241
min          0.239700
25%          0.366800
50%          0.435100
75%          0.544000
max          1.000000
Name: _max, dtype: float64

In [65]:
total_df.q3.describe()

count    68330.000000
mean         0.375842
std          0.093121
min          0.156300
25%          0.328700
50%          0.372600
75%          0.401200
max          1.000000
Name: q3, dtype: float64

In [66]:
total_df.mid.describe()

count    68330.000000
mean         0.334282
std          0.082003
min         -0.052800
25%          0.301000
50%          0.339900
75%          0.359500
max          0.999900
Name: mid, dtype: float64

In [67]:
total_df.q1.describe()

count    68330.000000
mean         0.297272
std          0.075527
min         -0.206500
25%          0.254925
50%          0.301800
75%          0.326600
max          0.999500
Name: q1, dtype: float64

In [68]:
total_df.describe()

,_max,q3,mid,q1,mean
count,68330.000000,68330.000000,68330.000000,68330.000000,68330.000000
mean,0.487768,0.375842,0.334282,0.297272,0.339243
std,0.184241,0.093121,0.082003,0.075527,0.077029
min,0.239700,0.156300,-0.052800,-0.206500,0.047889
25%,0.366800,0.328700,0.301000,0.254925,0.305452
50%,0.435100,0.372600,0.339900,0.301800,0.343320
75%,0.544000,0.401200,0.359500,0.326600,0.361801
max,1.000000,1.000000,0.999900,0.999500,0.999656


In [69]:
t_cid=[]
for i in range(len(total_df)):
    if(total_df['_max'][i]>= 0.5440 and total_df['q3'][i]>= 0.4012 and total_df['mid'][i]>=0.3595 and total_df['q1'][i]>=0.3266):
        print(total_df['cId'][i])
        t_cid.append(total_df['cId'][i])

eh18**
rlaehddms0**
dbwjd02**
gkssk98**
gaki90**
tjdalr**
love980**
lkim04**
did11**
kes08**
ahjeong**
dc10**
dawoon12**
godepsl**
wat02**
rladustn46**
rnlduatmdd**
dnfsd**
dbtjs6**
dnjswn**
116871220**
dhgpwl**
1826274754326984**
phj12**
hnkan**
ysg011**
leeda**
seebar19**
dbsgml3**
fpffkry**
kimmy68**
sdbsduds**
1013083382133743**
guswn973**
57775053**
kimow91**
leehj3**
via7**
kal83**
thsthd**
netfro**
ghj25**
rhtldd**
mmbb61**
dmsal28**
vip1**
wlddy**
bmy00**
qwe24**
wldbs3**
yubegot**
idlsw**
ekdls07**
wawa1**
qkrwjdn**
skdus16**
jgod46**
apfh**
qhal**
zx1000**
frem**
eeeee**
tlgus20**
tmdgml06**
melo**
yunjin**
ryusohy**
hongjin02**
wpdlaekdm**
mj3sun**
spamtotheh**
sksksk1**
sksms46**
injung32**
dmst**
qhfka2**
lys88**
lyj23**
gmlwls123**
yhj95**
kim0**
hin10**
53216660**
spjh4**
dd**
chzh900**
rkdms16**
dyrud08**
hjhfj**
tnwlsrkd12**
kbk04**
kje82**
sweey16**
kangi**
ghdwl**
jed11**
yku9**
qufzb**
enfh**
kokoa09**
hyein20**
tomeetb**
chic**
pinekis**
animee**
nadain**
taeu**
rc

whj02**
whekdms**
rlagpwl**
heh00**
pomme**
xqjung**
bb**
tndus26**
dbstnr01**
dptmx**
박지현
god9**
yong50**
15053126**
awesome**
siu0606**
lsmys1**
chgml6**
mukunga**
dbgksk17**
kkho700**
elf**
sh76**
qkrtlwls77**
tjwls24**
nara91**
psjpsj1**
dmsql21**
ydms**
kmj1**
dmsqls12**
bigbang16**
dahee14**
qor04**
znmz10**
lucy10**
sta26**
51685355**
yoona4**
dddong**
rkgml21**
ekftn66**
kmsu**
b2sti**
duslove7**
wlgo4**
vpdlsrns**
hmy69**
charm10**
vlakd**
660402097504348**
kar23**
rudalsdl**
daeungd03**
bell**
kmjloves**
kjk81**
dbsdud20**
ggj013**
buzz3**
yewon0**
kgh8**
dldbwjd49**
djyc06**
dpwls20**
n201112**
kczzz**
dkfma**
tndus05**
yeahjin**
okvla**
chwld**
wjdtj8**
andy62**
dmswn52**
cjstk64**
khr9**
joyshk72**
lh19**
rkfeo**
mij9**
gdgd88**
betsy**
chae**
lollipop**
김은지
haha94**
dkfmadld**
신지혜
ehdqkdal**
osy28409**
tjqhrud**
ppyj08**
boram**
m3m**
tnghk77**
wwkkll20**
tnwjd88**
52655962**
honglk6**
tix**
dltnwjd**
xmfpdls**
wtsd5**
jiyeong04**
tjsdud35**
hyeon9903**
gkswjdd**
aurora15

jbl070**
komh**
shakft**
iotnw**
eunchae03**
toffl01**
dbwjd29**
abc100**
ahnson**
rlarhdms**
hm0107**
guswl51**
anh**
jy9698**
rlatmdrud**
leemin85**
rlaalstj57**
thgml06**
ballet04**
daeun95**
wlsdsmtnr**
jun68**
tidhw**
alswl28**
ellen208**
jyy04**
wndus50**
girljae**
psa01**
dmswl9**
crave**
guswjdgml9**
mary22**
jhmin43**
wltot**
c49**
tjsgpd**
phj22**
lyj9908**
maei**
banggd**
oneda**
lyw99**
gyfla3**
rlawlsdl1**
wlgus70**
sohe**
lef**
gusdms**
adad33**
967714806638105**
lovecho**
songe2**
hyh25**
uhy88**
tjsrud02**
mn10**
hayerin**
qwqwqw37**
dogchi**
chooooi**
ke2k1j9**
tjstkcor1**
hello01a**
rkdwndus5**
bin49**
hwa86**
asongi5**
ekgml17**
kiju**
j199610**
jinh6**
xu**
clamp42**
seunghee**
park093**
alwjdrla2**
cjmb**
rkdms44**
dlgk98**
dmswl670**
alzl23**
73468207**
wntjsr**
jsua09**
sji**
dsec1**
jwy99**
54910193**
tnwls62**
kshksh01**
celen**
hooni1**
98793029**
jjang54**
anjiu017**
hyojin67**
meeso**
안은빛나
cjstkwldu**
2066461763623675**
dlekdms31**
이아영
11231853**
wlswn01**
s

In [70]:
print(len(t_cid))

3399


In [71]:
score_avg = []
for i in range(len(t_cid)):
    score_avg.append((go_data.loc[go_data['cId']==t_cid[i]])['rScore'].mean())

In [72]:
cnt_review = []

for i in range(len(cIds)):
    for j in range(len(t_cid)):
        if(cIds.index[i] == t_cid[j]):
            cnt_review.append(cIds['cId'][i])
            break;

In [73]:
final_df=pd.DataFrame(columns=['cId','cnt', 'score_avg'])
final_df['cId']=t_cid
final_df['cnt']=cnt_review
final_df['score_avg']=score_avg

final_df

,cId,cnt,score_avg
0,eh18**,85,5.000000
1,rlaehddms0**,83,4.987952
2,dbwjd02**,66,4.939394
3,gkssk98**,56,4.928571
4,gaki90**,51,5.000000
5,tjdalr**,50,4.920000
6,love980**,45,5.000000
7,lkim04**,43,5.000000
8,did11**,41,5.000000
9,kes08**,40,5.000000


In [74]:
print(len(t_cid))
print(len(cnt_review))
print(len(score_avg))

3399
3399
3399


In [75]:
final_df['score_avg'].mean()

4.871146347784206

In [76]:
final_df['cnt'].mean()

10.36510738452486

In [79]:
t_cid_df = pd.DataFrame(t_cid)
writer = pd.ExcelWriter('gogosing_doc2vec_new.xlsx')
t_cid_df.to_excel(writer,'Sheet1')
writer.save()

In [77]:
tagged_df = tokenizer_twitter_morphs('psbp**')
model = training_pvdbow(tagged_df)
sm = similarity(model)

/usr/local/lib/python2.7/dist-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Model Saved
최대값: 0.7516813265960658
상위25%: 0.4441848345350377
중간값: 0.3551477245906529
상위75%: 0.2598746166323791
평균: 0.3487942225983043


In [78]:
similar_doc = model.docvecs.most_similar('1')
print similar_doc

[('41', 0.6021578311920166), ('52', 0.574324369430542), ('26', 0.5420600771903992), ('8', 0.5409676432609558), ('46', 0.5051343441009521), ('6', 0.490250825881958), ('27', 0.4880243241786957), ('29', 0.48727723956108093), ('24', 0.4671546220779419), ('22', 0.45819905400276184)]


In [86]:
r.loc[1]

words    [데, 일리, 로, 입, 을, 흰, 블라우스, 찾다, 가, 샀, 어요, !, 저렴, 한, 가격, 에, 잘, 산, 거, 같아, 요, 스, 판도, 좋, 아서, 불편하지, 않아, 요, :)]
tags                                                                                                         [1]
Name: 1, dtype: object

In [87]:
r.loc[41]

words    [이뻐, 요, -, 키, ,, 몸무게, -, 착용, 사이즈, -, 착용, 후, 느낌, 은, ?(, 좋, 은, 점, /, 나쁜, 점, ), 화이트, 샀, 는데, 저렴, 한, 가격, 에, 잘산, 거, 같, 음, ㅋㅋ, 워터파크, 갈, 때, 비키니, 위, 에, 입으, 려고, 삿, 어요]
tags                                                                                                                                                              [41]
Name: 41, dtype: object

In [88]:
r.loc[52]

words    [빨강, 천, 이, 아닌, .., 스타킹, 가, 튼, ??, ㅎ, 암튼, 좋, 네, 욥, ㅎㅎㅎㅎㅎㅎ]
tags                                                          [52]
Name: 52, dtype: object

In [91]:
r.loc[26]

words    [포토, 후기, 3, 등, 이, 요, ㅎㅎ, 네이비, 색, 이뻐, 서, 샀, 는데, 엄마, 드렸, 어요, 이쁘, 데요, 제, 가, 보기, 에는, 산, 거중, 에, 이, 게, 제일, 이뻤, 어요, 하늘하늘, 해, 요, 그렇게, 많이, 비치, 지도, 않아, 요, 가슴, 부분, 이, 많이, 파, 인, 것, 같은, 데, 안, 에, 나시, 입고, 입어, 야, 할, 것, 같아, 요, ㅋㅋ, 예뻐, 요, 예뻥]
tags                                                                                                                                                                                                                                 [26]
Name: 26, dtype: object